# Decorators
## Author: Gustavo Amarante

Decorators provide a simple syntax for calling higher-order functions in Python. By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it (this sounds a little technical now, but keep going).



---
# Section 1 - Functions

This is something we covered already, but now we will see it in more detail.

### Example Function

In [1]:
def my_function():
    """
    Documentation
    """

If you ask what `my_function` is, it will return the function itself

In [2]:
my_function

<function __main__.my_function()>

If you put `()` at the end of the name of its name, it will **execute** the function (which in this case, does nothing).

In [3]:
my_function()

A function returns a value based on its given input arguments

In [4]:
def my_function(arg):
    """
    Documentation
    """
    
    my_var = arg * 5
    
    return my_var

In [5]:
my_function(3)

15

if nothing is passed as arguments, you get a type error

In [6]:
my_function()

TypeError: my_function() missing 1 required positional argument: 'arg'

A function can be passed to a variable, but remember that you must **not** put parenthesis in the end in this case.

In [7]:
also_my_function = my_function
also_my_function(3)

15

The key concept here is that **functions are first-class objects**. This means that **functions can also be used as arguments**, like any other first-class object (like strings, intergers, etc)

### Functions as First-Class Objects

In [8]:
def say_hello(name):
    return f"Hello {name}!"

def be_awsome(name):
    return f"Yo {name}, you are the best!"

In [9]:
say_hello('Gustavo')

'Hello Gustavo!'

Since functions also have the same properties of other first-class objects we can, for example, put them in a list.

In [10]:
my_list = [say_hello, be_awsome]
my_list[1]

<function __main__.be_awsome(name)>

In [11]:
my_list[1]('Fernando')

'Yo Fernando, you are the best!'

Now let us create a function that takes another function as an argument.

In [12]:
def greet_bob(greeter_function):
    return greeter_function("Bob")

In [13]:
greet_bob(say_hello)

'Hello Bob!'

In [14]:
greet_bob(be_awsome)

'Yo Bob, you are the best!'

### Inner Functions

This is a simple example to show that you can define functions inside of other functions.

In [15]:
def parent():
    print('Printing from the parent() function')
    
    def first_child():
        print('Printing from the first_child() function')
        
    def second_child():
        print('Printing from the second_child() function')
        
    first_child()
    second_child()

In [16]:
parent

<function __main__.parent()>

In [17]:
parent()

Printing from the parent() function
Printing from the first_child() function
Printing from the second_child() function


If you try to acess one of the inner functions you will get a name error. This functions actually reside **locally** in the `parent` function.

In [18]:
first_child

NameError: name 'first_child' is not defined

But there is a way to grab locally defined functions.

### Returning Functions from Functions

In [19]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"
    
    def second_child():
        return "Hello, I am Jake"
    
    if num == 1:
        return first_child
    else:
        return second_child

In [20]:
parent

<function __main__.parent(num)>

In [21]:
fisrt_child

NameError: name 'fisrt_child' is not defined

In [22]:
first = parent(1)
second = parent(2)

In [23]:
first

<function __main__.parent.<locals>.first_child()>

In [24]:
second

<function __main__.parent.<locals>.second_child()>

Notice that this functions are in `parent.<local>`

In [25]:
first()

'Hi, I am Emma'

In [26]:
second()

'Hello, I am Jake'

---
# Section 2 - Decorators
Take a look at the example below. What we want to do is to get a function that already exists `say_whee` and add something to happen before and after the function executed.

In [27]:
def my_decorator(func):
    def wrapper():
        print('Something is happening before the function is called.')
        func()
        print('Something is happening before the function is called.')
        
    return wrapper

def say_whee():
    print('Whee!!')

In [28]:
say_whee = my_decorator(say_whee)

In [29]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

In [30]:
say_whee()

Something is happening before the function is called.
Whee!!
Something is happening before the function is called.


**Decorators wrap a function**. They allow you to put some code around another function. (So they are literally a "decoration" around it)

In [31]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass
    return wrapper

def say_whee():
    print('Whee!!')

In [32]:
say_whee = not_during_the_night(say_whee)

In [33]:
say_whee()

Whee!!


Although we can create a decorator function and pass it another function as argument, we can use the beautiful python syntax to make things a little prettier. Let's repeat the example above, but now with the python syntax for decorators.

In [34]:
def my_decorator(func):
    
    def wrapper():
        
        print('Something is happening before the function is called.')
        func()
        print('Something is happening before the function is called.')
        
    return wrapper



@my_decorator
def say_whee():
    print('Whee!!')

In [35]:
say_whee()

Something is happening before the function is called.
Whee!!
Something is happening before the function is called.


So, all you have to do is to use `@my_decorator` right before defining a function in order to **wrap** it.

We can also pass arguments to decorator functions.

In [36]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f'Hello {name}')

In [37]:
greet('World')

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The function we are trying to use (`greet`) requeires an argument. But our wrapper does not accept it.

One thing we could do is to give an argument to the wrapper.

In [38]:
def do_twice(func):
    def wrapper_do_twice(arg1):
        func(arg1)
        func(arg1)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f'Hello {name}')
    
@do_twice
def say_whee():
    print('Whee!')

In [39]:
greet('World')

Hello World
Hello World


In [40]:
say_whee()

TypeError: wrapper_do_twice() missing 1 required positional argument: 'arg1'

Now the `say_whee` function is not working, because it does not take any arguments.

So what we really need to do is to allow for the decorator function to take any number of arguments. Here is how to do it:

In [41]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f'Hello {name}')
    
@do_twice
def say_whee():
    print('Whee!')

In [42]:
greet('World')

Hello World
Hello World


In [43]:
say_whee()

Whee!
Whee!


The term `*args` represents **arguments**. The term `*kwargs` represents **keyword arguments**. No matter how many there are, we can always pass as many arguments you want.

The next step is to return values from a decorator.

In [44]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        return func(*args, **kwargs)  # this is the important change. We are not returning a function, but it call
    return wrapper_do_twice


@do_twice
def greet(name):
    print(f'Hello {name}')
    
@do_twice
def say_whee():
    print('Whee!')

@do_twice
def return_greeting(name):
    print('creating greeting')
    return f'Hello {name}'

In [45]:
hi_adam = return_greeting("Adam")

creating greeting
creating greeting
creating greeting


In [46]:
hi_adam

'Hello Adam'

Think about about why `creating greeting` appeared 3 times during this execution.

Let us go through one last example before going to real world applications.

---
# Section 3 - Real World Examples

As you may have noticed already, decorator in python follow a similar pattern or a template.

The example below can be used for reference. Notice we added a `functools` method. All that it does is that it still maintains the original docunmention of the input function in the decorated function.

In [47]:
import functools

def decorator(func):
    
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):

        # Do something before
        
        value = func(*args, **kwargs)
        
        # Do something after
        
        return value
    return wrapper_decorator

### Timer Decorator
For this exemple let us create a **decorator that times how long functions take to run**.

In [48]:
import functools
import time

def timer(func):
    """Prints the runtime of the decorated function"""

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):

        start_time = time.perf_counter()
        
        value = func(*args, **kwargs)
        
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Run time for {func.__name__!r} was {run_time:.4f} seconds")
        
        return value
    return wrapper_timer

Now lets create a function that just takes a long time to run

In [49]:
@timer
def waste_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [50]:
waste_time(100)

Run time for 'waste_time' was 0.2884 seconds


### Debugging decorator

In [51]:
def debug(func):
    """print the function signature and return value"""
    
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):

        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k,v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        
        value = func(*args, **kwargs)
        
        print(f"{func.__name__!r} returned {value!r}")
        
        return value
    return wrapper_debug

In [52]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [53]:
make_greeting('Bob', 25)

Calling make_greeting('Bob', 25)
'make_greeting' returned 'Whoa Bob! 25 already, you are growing up!'


'Whoa Bob! 25 already, you are growing up!'

Here is another, more serious, example

In [54]:
import math

math.factorial = debug(math.factorial)  # Remember this syntax from ou early intuitions of decorators

def approx_e(terms=18):
    return sum(1/math.factorial(n) for n in range(terms))

In [55]:
approx_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333